In [ ]:
import face_recognition
import cv2
import numpy as np

# Đây là bản demo chạy nhận dạng khuôn mặt trên video trực tiếp từ webcam của bạn. Nó phức tạp hơn một chút so với
# ví dụ khác, nhưng nó bao gồm một số điều chỉnh hiệu suất cơ bản để giúp mọi thứ chạy nhanh hơn rất nhiều:
# 1. Xử lý từng khung hình video ở độ phân giải 1/4 (mặc dù vẫn hiển thị ở độ phân giải đầy đủ)
# 2. Chỉ phát hiện khuôn mặt ở mọi khung hình video khác.

# XIN LƯU Ý: Ví dụ này yêu cầu cài đặt OpenCV (thư viện `cv2`) chỉ để đọc từ webcam của bạn.
# OpenCV *không* bắt buộc phải sử dụng thư viện face_recognition. Nó chỉ được yêu cầu nếu bạn muốn chạy cái này
# bản demo cụ thể. Nếu bạn gặp khó khăn khi cài đặt nó, hãy thử bất kỳ bản demo nào khác không yêu cầu nó.

# Nhận tham chiếu đến webcam # 0 (mặc định)
video_capture = cv2.VideoCapture(0)

# Tải ảnh mẫu và tìm hiểu cách nhận biết nó.
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

# Tải ảnh mẫu thứ hai và tìm hiểu cách nhận dạng nó.
biden_image = face_recognition.load_image_file("biden.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

# Tạo mảng mã hóa khuôn mặt đã biết và tên của chúng
known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding
]
known_face_names = [
    "Barack Obama",
    "Joe Biden"
]

# Khởi tạo một số biến
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

while True:
    # Lấy một khung hình video
    ret, frame = video_capture.read()

    # Chỉ xử lý mọi khung hình video khác để tiết kiệm thời gian
    if process_this_frame:
        # Thay đổi kích thước khung hình của video thành kích thước 1/4 để xử lý nhận dạng khuôn mặt nhanh hơn
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

        # Chuyển đổi hình ảnh từ màu BGR (mà OpenCV sử dụng) sang màu RGB (sử dụng face_recognition)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        # Tìm tất cả các khuôn mặt và mã hóa khuôn mặt trong khung hình hiện tại của video
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # Xem khuôn mặt có khớp với (các) khuôn mặt đã biết hay không
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # # Nếu tìm thấy kết quả trùng khớp trong known_face_encodings, chỉ cần sử dụng kết quả đầu tiên.
            # nếu Đúng trong các trận đấu:
            # first_match_index =match.index(True)
            # tên = known_face_names[first_match_index]

            # Hoặc thay vào đó, sử dụng khuôn mặt đã biết với khoảng cách nhỏ nhất đến khuôn mặt mới
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            face_names.append(name)

    process_this_frame = not process_this_frame


    #Hiển thị kết quả
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Chia tỷ lệ sao lưu các vị trí khuôn mặt vì khung mà chúng tôi phát hiện được chia tỷ lệ thành kích thước 1/4
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Vẽ một cái hộp xung quanh khuôn mặt
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        ## Vẽ nhãn có tên bên dưới khuôn mặt
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Hiển thị hình ảnh kết quả
    cv2.imshow('Video', frame)

    # Nhấn 'q' trên bàn phím để thoát!
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Nhả tay cầm vào webcam
video_capture.release()
cv2.destroyAllWindows()